## Grid vs. random searches
## Methodology development

Grid search will be compared against random search based on their implementation using logistic regression and GBM as estimation methods. The tuning hyper-parameter of the first is *regularization parameter* $\lambda$, while the second method has the following hyper-parameters to be set: *subsample* $\eta$, *maximum depth* $J$, *learning rate* $v$ and *number of estimators* $M$.
<br>
<br>
Grid search for $\lambda$ will take place on the following set: $\Theta_{\lambda} = [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.25, 0.3, 0.5, 0.75, 1, 3, 10]$. For random search, its minimum and maximum values conceive the interval $(0.0001, 10)$ over which a three-modal random distribution will be defined: $Uniform(0.0001, 0.1)$, $Uniform(0.1, 1)$ and $Uniform(1, 10)$. A total of 10 random samples will be drawn, preserving the same density of values in each sub-interval $(0.0001, 0.1)$, $(0.1, 1)$ and $(1, 10)$ as found in $\Theta_{\lambda}$ - therefore, four random values from $(0.0001, 0.1)$, four from $(0.1, 1)$ and two from $(1, 10)$.
<br>
<br>
When it comes to the definition of $(\eta, J, v, M)$ for GBM estimation, grid search will look over $\Theta = \{0.75\}x\{1, 3, 5\}x\{0.0001, 0.01, 0.1\}x\{100, 250, 500\}$, so $|\Theta| = 27$. In order to keep things comparable, 20 random samples $(\eta, J, v, M)$ will be extracted based on the following distributions for each hyper-parameter: $\eta = 0.75$ will be kept constant, while $J \in \{1, 2, 3, 4, 5\}$ and $M \in \{100, 101, ..., 500\}$ will be defined from an ordinary random sampling. Finally, $v$ will come from $Uniform(0.0001, 0.1)$.

-----------

This notebook follows experiments design present in the first notebook of the series, and contains codes for implementing tests to compare grid and random searches. These tests, in their turn, will make use of Python scripts that generalize codes present here. A final notebook will assess and discuss results.

---------------

**Summary:**
1. [Libraries](#libraries)<a href='#libraries'></a>.
2. [Functions and classes](#functions_classes)<a href='#functions_classes'></a>.
3. [Settings](#settings)<a href='#settings'></a>.
4. [Importing data](#imports)<a href='#imports'></a>.
    * [Categorical features](#categorical_features)<a href='#categorical_features'></a>.
    * [Model assessment](#model_assessment)<a href='#model_assessment'></a>.
    * [Classifying features](#classif_feat)<a href='#classif_feat'></a>.
<br>
<br>
5. [Data pre-processing](#data_pre_proc)<a href='#data_pre_proc'></a>.
    * [Assessing missing values](#assessing_missing)<a href='#assessing_missing'></a>.
    * [Transforming numerical features](#num_transf)<a href='#num_transf'></a>.
    * [Transforming categorical features](#categorical_transf)<a href='#categorical_transf'></a>.
    * [Datasets structure](#datasets_structure)<a href='#datasets_structure'></a>.
<br>
<br>
6. [Model estimation](#model_estimation)<a href='#model_estimation'></a>.
    * [Grids of hyper-parameters](#grids)<a href='#grids'></a>.
    * [Estimations](#estimations)<a href='#estimations'></a>.
<br>
<br>
7. [Assessment of results](#assess_results)<a href='#assess_results'></a>.

<a id='libraries'></a>

## Libraries

In [1]:
import pandas as pd
import numpy as np
import json
import os

from datetime import datetime
import time

import progressbar
from time import sleep

from scipy.stats import uniform, norm, randint

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, auc, precision_recall_curve, brier_score_loss

<a id='functions_classes'></a>

## Functions and classes

In [2]:
import utils
from utils import epoch_to_date, text_clean, is_velocity, get_cat

In [3]:
from transformations import log_transformation, standard_scale, recreate_missings, impute_missing
from transformations import one_hot_encoding

In [4]:
import validation
from validation import Kfolds_fit

<a id='settings'></a>

## Settings

### Files management

In [5]:
estimation_id = str(int(time.time()))
start_time = datetime.now()

# Declare whether to export results:
export = False

# Define a dataset id:
s = 9098

### Numerical data transformation

In [6]:
# Declare whether to apply logarithmic transformation over numerical data:
log_transform = True

# Declare whether to standardize numerical data:
standardize = True

### Model estimation

In [7]:
# Select an estimation method ['logistic_regression', 'GBM']:
method = 'logistic_regression'

# Choose whether to perform random search (True) or grid search (False):
random_search = False

# Define the number of samples to implement random search:
if method == 'logistic_regression':
    n_samples = 10

elif method == 'GBM':
    n_samples = 20

<a id='imports'></a>

## Importing data

In [8]:
# Train data:
os.chdir('/home/matheus_rosso/Arquivo/Features/Datasets/')

df_train = pd.read_csv('new_additional_datasets/dataset_' + str(s) + '.csv',
                       dtype={'order_id': str, 'store_id': int})
df_train.drop_duplicates(['order_id', 'epoch', 'order_amount'], inplace=True)
df_train['date'] = df_train.epoch.apply(epoch_to_date)

# Dropping original categorical features:
cat_vars = get_cat(df_train)
c_vars = [c for c in list(df_train.columns) if 'C#' in c]
na_vars = ['NA#' + c for c in cat_vars if 'NA#' + c in list(df_train.columns)]

df_train = df_train.drop(c_vars, axis=1).drop(na_vars, axis=1)

# Splitting data into train and test:
df_test = df_train[(df_train.date > datetime.strptime('2020-03-30', '%Y-%m-%d'))]
df_train = df_train[(df_train.date <= datetime.strptime('2020-03-30', '%Y-%m-%d'))]

print('\033[1mShape of df_train for store ' + str(s) + ':\033[0m ' + str(df_train.shape) + '.')
print('\033[1mShape of df_test for store ' + str(s) + ':\033[0m ' + str(df_test.shape) + '.')
print('\n')

# Accessory variables:
drop_vars = ['y', 'order_amount', 'store_id', 'order_id', 'status', 'epoch', 'date', 'weight']

df_train.head(3)

Shape of df_train for store 9098: (7520, 2260).
Shape of df_test for store 9098: (11218, 2260).




,BILLINGLARGEAREAREPUTATION(),BILLINGSMALLAREAREPUTATION(),"BILLINGZIP(CREDITCARD,10080)","BILLINGZIP(CREDITCARD,1440)","BILLINGZIP(CREDITCARD,21600)","BILLINGZIP(CREDITCARD,360)","BILLINGZIP(CREDITCARD,43200)","BILLINGZIP(CREDITCARD,60)","BILLINGZIP(CREDITCARD,64800)","BILLINGZIP(DOCUMENT,10080)",...,ZIPFIRST3REPUTATION(),ZIPFIRST5REPUTATION(),y,order_amount,order_id,status,epoch,store_id,weight,date
0,0.022820,0.008911,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,...,0.025636,0.0,0.0,271.39,449e1d8e-63aa-4f04-a865-a5559db83d1a,APPROVED,1.577894e+12,9098,1.0,2020-01-01
1,0.024314,0.007161,1.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,...,0.027638,0.0,0.0,113.84,248d5388-83de-468f-ad04-641492f16b22,APPROVED,1.577894e+12,9098,1.0,2020-01-01
2,0.054615,0.006761,2.0,1.0,2.0,1.0,3.0,1.0,3.0,1.0,...,0.012398,0.0,0.0,179.86,ab36eee7-ee22-4fcd-abaf-4437ec534d1d,APPROVED,1.577914e+12,9098,1.0,2020-01-01


In [9]:
# Assessing missing values:
num_miss_train = df_train.isnull().sum().sum()
num_miss_test = df_test.isnull().sum().sum()

if num_miss_train > 0:
    print('\033[1mProblem - Number of overall missings detected (training data):\033[0m ' +
          str(df_train.isnull().sum().sum()) + '.')
    print('\n')

if num_miss_test > 0:
    print('\033[1mProblem - Number of overall missings detected (test data):\033[0m ' +
          str(df_test.isnull().sum().sum()) + '.')
    print('\n')

<a id='categorical_features'></a>

### Categorical features

In [10]:
categorical_train = pd.read_csv('new_additional_datasets/categorical_features/dataset_' + str(s) + '.csv',
                      dtype={'order_id': str, 'store_id': int})
categorical_train.drop_duplicates(['order_id', 'epoch', 'order_amount'], inplace=True)

categorical_train['date'] = categorical_train.epoch.apply(epoch_to_date)

# Splitting data into train and test:
categorical_test = categorical_train[(categorical_train.date > datetime.strptime('2020-03-30', '%Y-%m-%d'))]
categorical_train = categorical_train[(categorical_train.date <= datetime.strptime('2020-03-30', '%Y-%m-%d'))]

print('\033[1mShape of categorical_train (training data):\033[0m ' + str(categorical_train.shape) + '.')
print('\033[1mNumber of orders (training data):\033[0m ' + str(categorical_train.order_id.nunique()) + '.')
print('\n')

print('\033[1mShape of categorical_test (test data):\033[0m ' + str(categorical_test.shape) + '.')
print('\033[1mNumber of orders (test data):\033[0m ' + str(categorical_test.order_id.nunique()) + '.')
print('\n')

categorical_train.head()

Shape of categorical_train (training data): (7520, 22).
Number of orders (training data): 7520.


Shape of categorical_test (test data): (11218, 22).
Number of orders (test data): 11218.




,BILLINGCITY(),BILLINGSTATE(),BROWSER(),CREDITCARDBRAND(),CREDITCARDCOUNTRY(),CREDITCARDSUBTYPE(),EMAILDOMAIN(),GENDERBYNAMEPTBR(),IPGEOLOCATIONCITY(),IPGEOLOCATIONCOUNTRY(),...,SHIPPINGSTATE(),UTMSOURCELASTCLICK(),y,order_amount,order_id,status,epoch,store_id,weight,date
0,São Paulo,SP,Chrome Mobile,MASTERCARD,BR,BLACK,bancotoyota.com.br,F,São Paulo,BR,...,SP,NaN,0.0,271.39,449e1d8e-63aa-4f04-a865-a5559db83d1a,APPROVED,1.577894e+12,9098,1.0,2020-01-01
1,São Paulo,SP,Chrome Mobile,MASTERCARD,BR,GOLD,gmail.com,F,São Paulo,BR,...,SP,NaN,0.0,113.84,248d5388-83de-468f-ad04-641492f16b22,APPROVED,1.577894e+12,9098,1.0,2020-01-01
2,São Paulo,SP,Mobile Safari,VISA,BR,PLATINUM,recoder.com.br,M,São Paulo,BR,...,SP,NaN,0.0,179.86,ab36eee7-ee22-4fcd-abaf-4437ec534d1d,APPROVED,1.577914e+12,9098,1.0,2020-01-01
3,São Paulo,SP,Chrome,MASTERCARD,BR,GOLD,gmail.com,F,São Paulo,BR,...,SP,NaN,0.0,68.45,34f6e7d5-eef6-4a2b-9ba5-2c3c2e9be47e,APPROVED,1.577965e+12,9098,1.0,2020-01-02
4,São Paulo,SP,Chrome,VISA,BR,PLATINUM,gmail.com,F,São Paulo,BR,...,SP,NaN,0.0,302.30,4ecf9829-62a3-499a-ad08-fc1f858f2e01,APPROVED,1.577970e+12,9098,1.0,2020-01-02


#### Treating missing values

In [11]:
print('\033[1mAssessing missing values in categorical data (training data):\033[0m')
print(categorical_train.drop(drop_vars, axis=1).isnull().sum().sort_values(ascending=False))

Assessing missing values in categorical data (training data):
UTMSOURCELASTCLICK()      7114
CREDITCARDSUBTYPE()       4859
CREDITCARDCOUNTRY()       4842
CREDITCARDBRAND()         4842
IPGEOLOCATIONCITY()        122
IPGEOLOCATIONCOUNTRY()      77
BROWSER()                   77
GENDERBYNAMEPTBR()           2
SHIPPINGSTATE()              0
SHIPPINGCITY()               0
SELLERID()                   0
EMAILDOMAIN()                0
BILLINGSTATE()               0
BILLINGCITY()                0
dtype: int64


In [12]:
print('\033[1mAssessing missing values in categorical data (test data):\033[0m')
print(categorical_test.drop(drop_vars, axis=1).isnull().sum().sort_values(ascending=False))

Assessing missing values in categorical data (test data):
CREDITCARDSUBTYPE()       11218
CREDITCARDCOUNTRY()       11218
CREDITCARDBRAND()         11218
UTMSOURCELASTCLICK()      10687
IPGEOLOCATIONCITY()         358
IPGEOLOCATIONCOUNTRY()       87
BROWSER()                    73
GENDERBYNAMEPTBR()            6
SHIPPINGSTATE()               0
SHIPPINGCITY()                0
SELLERID()                    0
EMAILDOMAIN()                 0
BILLINGSTATE()                0
BILLINGCITY()                 0
dtype: int64


In [13]:
# Loop over categorical features:
for f in categorical_train.drop(drop_vars, axis=1).columns:
    # Training data
    categorical_train[f] = categorical_train[f].apply(lambda x: 'NA_VALUE' if pd.isna(x) else x)
    
    # Test data:
    categorical_test[f] = categorical_test[f].apply(lambda x: 'NA_VALUE' if pd.isna(x) else x)

In [14]:
# Assessing missing values:
if categorical_train.isnull().sum().sum() > 0:
    print('\033[1mProblem - Number of overall missings detected (training data):\033[0m ' +
          str(categorical_train.isnull().sum().sum()) + '.')
    print('\n')

if categorical_test.isnull().sum().sum() > 0:
    print('\033[1mProblem - Number of overall missings detected (test data):\033[0m ' +
          str(categorical_test.isnull().sum().sum()) + '.')
    print('\n')

#### Treating text data

In [15]:
na_vars = [c for c in categorical_train.drop(drop_vars, axis=1) if 'NA#' in c]

# Loop over categorical features:
for f in categorical_train.drop(drop_vars, axis=1).drop(na_vars, axis=1).columns:
    # Training data:
    categorical_train[f] = categorical_train[f].apply(lambda x: text_clean(str(x)))
    
    # Test data:
    categorical_test[f] = categorical_test[f].apply(lambda x: text_clean(str(x)))

categorical_train.head()

,BILLINGCITY(),BILLINGSTATE(),BROWSER(),CREDITCARDBRAND(),CREDITCARDCOUNTRY(),CREDITCARDSUBTYPE(),EMAILDOMAIN(),GENDERBYNAMEPTBR(),IPGEOLOCATIONCITY(),IPGEOLOCATIONCOUNTRY(),...,SHIPPINGSTATE(),UTMSOURCELASTCLICK(),y,order_amount,order_id,status,epoch,store_id,weight,date
0,sao_paulo,sp,chrome_mobile,mastercard,br,black,bancotoyota.com.br,f,sao_paulo,br,...,sp,na_value,0.0,271.39,449e1d8e-63aa-4f04-a865-a5559db83d1a,APPROVED,1.577894e+12,9098,1.0,2020-01-01
1,sao_paulo,sp,chrome_mobile,mastercard,br,gold,gmail.com,f,sao_paulo,br,...,sp,na_value,0.0,113.84,248d5388-83de-468f-ad04-641492f16b22,APPROVED,1.577894e+12,9098,1.0,2020-01-01
2,sao_paulo,sp,mobile_safari,visa,br,platinum,recoder.com.br,m,sao_paulo,br,...,sp,na_value,0.0,179.86,ab36eee7-ee22-4fcd-abaf-4437ec534d1d,APPROVED,1.577914e+12,9098,1.0,2020-01-01
3,sao_paulo,sp,chrome,mastercard,br,gold,gmail.com,f,sao_paulo,br,...,sp,na_value,0.0,68.45,34f6e7d5-eef6-4a2b-9ba5-2c3c2e9be47e,APPROVED,1.577965e+12,9098,1.0,2020-01-02
4,sao_paulo,sp,chrome,visa,br,platinum,gmail.com,f,sao_paulo,br,...,sp,na_value,0.0,302.30,4ecf9829-62a3-499a-ad08-fc1f858f2e01,APPROVED,1.577970e+12,9098,1.0,2020-01-02


#### Merging all features

In [16]:
# Training data:
df_train = df_train.merge(categorical_train[[f for f in categorical_train.columns if (f not in drop_vars) |
                                             (f == 'order_id')]],
                          on='order_id', how='left')

print('\033[1mShape of df_train for store ' + str(s) + ':\033[0m ' + str(df_train.shape) + '.')
print('\n')

df_train.head()

Shape of df_train for store 9098: (7520, 2274).




,BILLINGLARGEAREAREPUTATION(),BILLINGSMALLAREAREPUTATION(),"BILLINGZIP(CREDITCARD,10080)","BILLINGZIP(CREDITCARD,1440)","BILLINGZIP(CREDITCARD,21600)","BILLINGZIP(CREDITCARD,360)","BILLINGZIP(CREDITCARD,43200)","BILLINGZIP(CREDITCARD,60)","BILLINGZIP(CREDITCARD,64800)","BILLINGZIP(DOCUMENT,10080)",...,CREDITCARDCOUNTRY(),CREDITCARDSUBTYPE(),EMAILDOMAIN(),GENDERBYNAMEPTBR(),IPGEOLOCATIONCITY(),IPGEOLOCATIONCOUNTRY(),SELLERID(),SHIPPINGCITY(),SHIPPINGSTATE(),UTMSOURCELASTCLICK()
0,0.022820,0.008911,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,...,br,black,bancotoyota.com.br,f,sao_paulo,br,none,sao_paulo,sp,na_value
1,0.024314,0.007161,1.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,...,br,gold,gmail.com,f,sao_paulo,br,none,sao_paulo,sp,na_value
2,0.054615,0.006761,2.0,1.0,2.0,1.0,3.0,1.0,3.0,1.0,...,br,platinum,recoder.com.br,m,sao_paulo,br,none,sao_paulo,sp,na_value
3,0.035726,0.009156,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,br,gold,gmail.com,f,sao_paulo,br,none,sao_paulo,sp,na_value
4,0.049755,0.018226,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,...,br,platinum,gmail.com,f,sao_paulo,br,none,sao_paulo,sp,na_value


In [17]:
# Test data:
df_test = df_test.merge(categorical_test[[f for f in categorical_test.columns if (f not in drop_vars) |
                                          (f == 'order_id')]],
                        on='order_id', how='left')

print('\033[1mShape of df_test for store ' + str(s) + ':\033[0m ' + str(df_test.shape) + '.')
print('\n')

df_test.head()

Shape of df_test for store 9098: (11218, 2274).




,BILLINGLARGEAREAREPUTATION(),BILLINGSMALLAREAREPUTATION(),"BILLINGZIP(CREDITCARD,10080)","BILLINGZIP(CREDITCARD,1440)","BILLINGZIP(CREDITCARD,21600)","BILLINGZIP(CREDITCARD,360)","BILLINGZIP(CREDITCARD,43200)","BILLINGZIP(CREDITCARD,60)","BILLINGZIP(CREDITCARD,64800)","BILLINGZIP(DOCUMENT,10080)",...,CREDITCARDCOUNTRY(),CREDITCARDSUBTYPE(),EMAILDOMAIN(),GENDERBYNAMEPTBR(),IPGEOLOCATIONCITY(),IPGEOLOCATIONCOUNTRY(),SELLERID(),SHIPPINGCITY(),SHIPPINGSTATE(),UTMSOURCELASTCLICK()
0,0.034498,0.013024,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,...,na_value,na_value,gmail.com,m,sao_paulo,br,13,sao_paulo,sp,na_value
1,0.030711,0.007754,2.0,1.0,5.0,1.0,5.0,1.0,5.0,2.0,...,na_value,na_value,gmail.com,f,sao_paulo,br,35,sao_paulo,sp,na_value
2,0.027144,0.011135,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,...,na_value,na_value,yahoo.com.br,m,sao_paulo,br,13,sao_paulo,sp,na_value
3,0.050302,0.005572,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,na_value,na_value,yahoo.com.br,f,richmond,gb,13,sao_paulo,sp,na_value
4,0.002414,0.002414,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,na_value,na_value,gmail.com,m,sao_paulo,br,13,sao_paulo,sp,na_value


In [18]:
# Assessing missing values (training data):
if df_train.isnull().sum().sum() != num_miss_train:
    print('\033[1mInconsistent number of overall missings values (training data)!\033[0m')
    print('\n')

# Assessing missing values (test data):
if df_test.isnull().sum().sum() != num_miss_test:
    print('\033[1mInconsistent number of overall missings values (test data)!\033[0m')
    print('\n')

<a id='model_assessment'></a>

### Model assessment

In [19]:
# Dictionary with information on model structure and performance:
os.chdir('/home/matheus_rosso/Arquivo/Materiais/Codes/grid_random_searches/')

if 'model_assessment.json' not in os.listdir('Datasets'):
    model_assessment = {}

else:
    with open('Datasets/model_assessment.json') as json_file:
        model_assessment = json.load(json_file)

<a id='classif_feat'></a>

### Classifying features

In [20]:
# Categorical features:
cat_vars = list(categorical_train.drop(drop_vars, axis=1).columns)

# Dummy variables indicating missing value status:
missing_vars = [c for c in list(df_train.drop(drop_vars, axis=1).columns) if ('NA#' in c)]

# Dropping features with no variance:
no_variance = [c for c in df_train.drop(drop_vars, axis=1).drop(cat_vars,
                                                                axis=1).drop(missing_vars,
                                                                             axis=1) if df_train[c].var()==0]

if len(no_variance) > 0:
    df_train.drop(no_variance, axis=1, inplace=True)
    df_test.drop(no_variance, axis=1, inplace=True)

# Numerical features:
cont_vars = [c for c in  list(df_train.drop(drop_vars, axis=1).columns) if is_velocity(c)]

# Binary features:
binary_vars = [c for c in list(df_train.drop([c for c in df_train.columns if (c in drop_vars) |
                                             (c in cat_vars) | (c in missing_vars) | (c in cont_vars)],
                                             axis=1).columns) if set(df_train[c].unique()) == set([0,1])]

# Updating the list of numerical features:
for c in list(df_train.drop(drop_vars, axis=1).columns):
    if (c not in cat_vars) & (c not in missing_vars) & (c not in cont_vars) & (c not in binary_vars):
        cont_vars.append(c)

# Dataframe presenting the frequency of features by class:
feats_assess = pd.DataFrame(data={
    'class': ['cat_vars', 'missing_vars', 'binary_vars', 'cont_vars', 'drop_vars'],
    'frequency': [len(cat_vars), len(missing_vars), len(binary_vars), len(cont_vars), len(drop_vars)]
})
feats_assess.sort_values('frequency', ascending=False)

,class,frequency
3,cont_vars,1551
1,missing_vars,480
2,binary_vars,34
0,cat_vars,14
4,drop_vars,8


<a id='data_pre_proc'></a>

## Data pre-processing

<a id='assessing_missing'></a>

### Assessing missing values

#### Recreating missing values

In [21]:
missing_vars = [f for f in df_train.columns if 'NA#' in f]

# Loop over variables with missing values:
for f in [c for c in missing_vars if c.replace('NA#', '') not in cat_vars]:
    if f.replace('NA#', '') in df_train.columns:
        # Training data:
        df_train[f.replace('NA#', '')] = recreate_missings(df_train[f.replace('NA#', '')], df_train[f])
        
        # Test data:
        df_test[f.replace('NA#', '')] = recreate_missings(df_test[f.replace('NA#', '')], df_test[f])
    else:
        df_train.drop([f], axis=1, inplace=True)
        
        df_test.drop([f], axis=1, inplace=True)

In [22]:
# Dropping all variables with missing value status:
df_train.drop([f for f in df_train.columns if 'NA#' in f], axis=1, inplace=True)

df_test.drop([f for f in df_test.columns if 'NA#' in f], axis=1, inplace=True)

#### Describing the frequency of missing values

In [23]:
# Dataframe with the number of missings by feature (training data):
missings_dict = df_train.isnull().sum().sort_values(ascending=False).to_dict()

missings_assess_train = pd.DataFrame(data={
    'feature': list(missings_dict.keys()),
    'missings': list(missings_dict.values())
})

print('\033[1mNumber of features with missings:\033[0m {}'.format(sum(missings_assess_train.missings > 0)) +
      ' out of {} features'.format(len(missings_assess_train)) +
      ' ({}%).'.format(round((sum(missings_assess_train.missings > 0)/len(missings_assess_train))*100, 2)))
print('\033[1mAverage number of missings:\033[0m {}'.format(int(missings_assess_train.missings.mean())) +
      ' out of {} observations'.format(len(df_train)) +
      ' ({}%).'.format(round((int(missings_assess_train.missings.mean())/len(df_train))*100,2)))
print('\n')

missings_assess_train.index.name = 'training_data'
missings_assess_train.head(10)

Number of features with missings: 460 out of 1607 features (28.62%).
Average number of missings: 1395 out of 7520 observations (18.55%).




,feature,missings
training_data,,
0,"USRNAVCOUNT(9,1h)",7409
1,"GFINGERPRINT(TOTAL_AMOUNT,60)",7408
2,"CUSTNAVCOUNT(ta,30min)",7396
3,"FINGERPRINT(TOTAL_AMOUNT,360)",7390
4,"GTELEPHONE(TOTAL_AMOUNT,60)",7385
5,"NAME(TOTAL_AMOUNT,360)",7382
6,"USRNAVCOUNT(ta,30min)",7381
7,"EMAIL(TOTAL_AMOUNT,360)",7381
8,"CUSTNAVCOUNT(ta,1h)",7379


In [24]:
# Dataframe with the number of missings by feature (test data):
missings_dict = df_test.isnull().sum().sort_values(ascending=False).to_dict()

missings_assess_test = pd.DataFrame(data={
    'feature': list(missings_dict.keys()),
    'missings': list(missings_dict.values())
})

print('\033[1mNumber of features with missings:\033[0m {}'.format(sum(missings_assess_test.missings > 0)) +
      ' out of {} features'.format(len(missings_assess_test)) +
      ' ({}%).'.format(round((sum(missings_assess_test.missings > 0)/len(missings_assess_test))*100, 2)))
print('\033[1mAverage number of missings:\033[0m {}'.format(int(missings_assess_test.missings.mean())) +
      ' out of {} observations'.format(len(df_test)) +
      ' ({}%).'.format(round((int(missings_assess_test.missings.mean())/len(df_test))*100,2)))
print('\n')
missings_assess_test.index.name = 'test_data'
missings_assess_test.head(10)

Number of features with missings: 460 out of 1607 features (28.62%).
Average number of missings: 2657 out of 11218 observations (23.69%).




,feature,missings
test_data,,
0,"SHIPPING_NAME(IP,43200)",11218
1,"SHIPPING_NAME(FINGERPRINT,1440)",11218
2,"SHIPPING_NAME(EMAIL,10080)",11218
3,"SHIPPING_NAME(EMAIL,1440)",11218
4,"SHIPPING_NAME(EMAIL,21600)",11218
5,"SHIPPING_NAME(EMAIL,360)",11218
6,"SHIPPING_NAME(EMAIL,43200)",11218
7,"SHIPPING_NAME(EMAIL,60)",11218
8,"SHIPPING_NAME(EMAIL,64800)",11218


<a id='num_transf'></a>

### Transforming numerical features

#### Logarithmic transformation

In [25]:
print('---------------------------------------------------------------------------------------------------------')
print('\033[1mAPPLYING LOGARITHMIC TRANSFORMATION OVER NUMERICAL DATA\033[0m')
print('\n')
# Variables that should not be log-transformed:
not_log = [c for c in df_train.columns if c not in cont_vars]

if log_transform:
    print('\033[1mTraining data:\033[0m')

    # Assessing missing values (before logarithmic transformation):
    num_miss_train = df_train.isnull().sum().sum()
    if num_miss_train > 0:
        print('\033[1mNumber of overall missings detected (before logarithmic transformation):\033[0m ' +
              str(num_miss_train) + '.')

    log_transf = log_transformation(not_log=not_log)
    log_transf.transform(df_train)
    df_train = log_transf.log_transformed

    # Assessing missing values (after logarithmic transformation):
    num_miss_train_log = df_train.isnull().sum().sum()
    if num_miss_train_log > 0:
        print('\033[1mNumber of overall missings detected (after logarithmic transformation):\033[0m ' + 
              str(num_miss_train_log) + '.')

    # Checking consistency in the number of missings:
    if num_miss_train_log != num_miss_train:
        print('\033[1mProblem - Inconsistent number of overall missings!\033[0m')

    print('\n')
    print('\033[1mTest data:\033[0m')

    # Assessing missing values (before logarithmic transformation):
    num_miss_test = df_test.isnull().sum().sum()
    if num_miss_test > 0:
        print('\033[1mNumber of overall missings detected (before logarithmic transformation):\033[0m ' +
              str(num_miss_test) + '.')

    log_transf = log_transformation(not_log=not_log)
    log_transf.transform(df_test)
    df_test = log_transf.log_transformed

    # Assessing missing values (after logarithmic transformation):
    num_miss_test_log = df_test.isnull().sum().sum()
    if num_miss_test_log > 0:
        print('\033[1mNumber of overall missings detected (after logarithmic transformation):\033[0m ' + 
              str(num_miss_test_log) + '.')

    # Checking consistency in the number of missings:
    if num_miss_test_log != num_miss_test:
        print('\033[1mProblem - Inconsistent number of overall missings!\033[0m')

else:
    print('\033[1mNo transformation performed!\033[0m')

print('\n')
print('---------------------------------------------------------------------------------------------------------')
print('\n')

---------------------------------------------------------------------------------------------------------
APPLYING LOGARITHMIC TRANSFORMATION OVER NUMERICAL DATA


Training data:
Number of overall missings detected (before logarithmic transformation): 2242917.
Number of numerical variables log-transformed: 1551.
Number of overall missings detected (after logarithmic transformation): 2242917.


Test data:
Number of overall missings detected (before logarithmic transformation): 4271178.
Number of numerical variables log-transformed: 1551.
Number of overall missings detected (after logarithmic transformation): 4271178.


---------------------------------------------------------------------------------------------------------




#### Standardizing numerical features

In [26]:
print('---------------------------------------------------------------------------------------------------------')
print('\033[1mAPPLYING STANDARD SCALE TRANSFORMATION OVER NUMERICAL DATA\033[0m')
print('\n')
# Inputs that should not be standardized:
not_stand = [c for c in df_train.columns if c.replace('L#', '') not in cont_vars]

if standardize:
    print('\033[1mTraining data:\033[0m')

    stand_scale = standard_scale(not_stand = not_stand)
    
    stand_scale.scale(train = df_train, test = df_test)
    
    df_train_scaled = stand_scale.train_scaled
    print('\033[1mShape of df_train_scaled (after scaling):\033[0m ' + str(df_train_scaled.shape) + '.')

    # Assessing missing values (after standardizing numerical features):
    num_miss_train = df_train.isnull().sum().sum()
    num_miss_train_scaled = df_train_scaled.isnull().sum().sum()
    if num_miss_train_scaled > 0:
        print('\033[1mNumber of overall missings:\033[0m ' + str(num_miss_train_scaled) + '.')
    else:
        print('\033[1mNo missing values detected (training data)!\033[0m')

    if num_miss_train_scaled != num_miss_train:
        print('\033[1mProblem - Inconsistent number of overall missings!\033[0m')
    
    print('\n')
    print('\033[1mTest data:\033[0m')
    df_test_scaled = stand_scale.test_scaled
    print('\033[1mShape of df_test_scaled (after scaling):\033[0m ' + str(df_test_scaled.shape) + '.')

    # Assessing missing values (after standardizing numerical features):
    num_miss_test = df_test.isnull().sum().sum()
    num_miss_test_scaled = df_test_scaled.isnull().sum().sum()
    if num_miss_test_scaled > 0:
        print('\033[1mNumber of overall missings:\033[0m ' + str(num_miss_test_scaled) + '.')
    else:
        print('\033[1mNo missing values detected (test data)!\033[0m')

    if num_miss_test_scaled != num_miss_test:
        print('\033[1mProblem - Inconsistent number of overall missings!\033[0m')

else:
    df_train_scaled = df_train.copy()
    df_test_scaled = df_test.copy()
    
    print('\033[1mNo transformation performed!\033[0m')

print('\n')
print('---------------------------------------------------------------------------------------------------------')
print('\n')

---------------------------------------------------------------------------------------------------------
APPLYING STANDARD SCALE TRANSFORMATION OVER NUMERICAL DATA


Training data:
Shape of df_train_scaled (after scaling): (7520, 1607).
Number of overall missings: 2242917.


Test data:
Shape of df_test_scaled (after scaling): (11218, 1607).
Number of overall missings: 4271178.


---------------------------------------------------------------------------------------------------------




#### Treating missing values

In [27]:
print('---------------------------------------------------------------------------------------------------------')
print('\033[1mTREATING MISSING VALUES\033[0m')
print('\n')

print('\033[1mTraining data:\033[0m')
num_miss_train = df_train_scaled.isnull().sum().sum()
print('\033[1mNumber of overall missing values detected before treatment:\033[0m ' +
      str(num_miss_train) + '.')

# Loop over features:
for f in df_train_scaled.drop(drop_vars, axis=1):
    # Checking if there is missing values for a given feature:
    if df_train_scaled[f].isnull().sum() > 0:
        check_missing = impute_missing(df_train_scaled[f])
        df_train_scaled[f] = check_missing['var']
        df_train_scaled['NA#' + f.replace('L#', '')] = check_missing['missing_var']

num_miss_train_treat = int(sum([sum(df_train_scaled[f]) for f in df_train_scaled.columns if 'NA#' in f]))
print('\033[1mNumber of overall missing values detected during treatment:\033[0m ' +
      str(num_miss_train_treat) + '.')

if num_miss_train_treat != num_miss_train:
    print('\033[1mProblem - Inconsistent number of overall missings!\033[0m')

if df_train_scaled.isnull().sum().sum() > 0:
    print('\033[1mProblem - Number of overall missings detected (training data):\033[0m ' +
          str(df_train_scaled.isnull().sum().sum()) + '.')

print('\n')
print('\033[1mTest data:\033[0m')
num_miss_test = df_test_scaled.isnull().sum().sum()
num_miss_test_treat = 0
print('\033[1mNumber of overall missing values detected before treatment:\033[0m ' + str(num_miss_test) + '.')

# Loop over features:
for f in df_test_scaled.drop(drop_vars, axis=1):
    # Check if there is dummy variable of missing value status for training data:
    if 'NA#' + f.replace('L#', '') in list(df_train_scaled.columns):
        check_missing = impute_missing(df_test_scaled[f])
        df_test_scaled[f] = check_missing['var']
        df_test_scaled['NA#' + f.replace('L#', '')] = check_missing['missing_var']
    else:
        # Checking if there are missings for variables without missings in training data:
        if df_test_scaled[f].isnull().sum() > 0:
            num_miss_test_treat += df_test_scaled[f].isnull().sum()
            df_test_scaled[f].fillna(0, axis=0, inplace=True)

num_miss_test_treat += int(sum([sum(df_test_scaled[f]) for f in df_test_scaled.columns if 'NA#' in f]))
print('\033[1mNumber of overall missing values detected during treatment:\033[0m ' +
      str(num_miss_test_treat) + '.')

if num_miss_test_treat != num_miss_test:
    print('\033[1mProblem - Inconsistent number of overall missings!\033[0m')

if df_test_scaled.isnull().sum().sum() > 0:
    print('\033[1mProblem - Number of overall missings detected (test data):\033[0m ' +
          str(df_test_scaled.isnull().sum().sum()) + '.')

print('\n')
print('---------------------------------------------------------------------------------------------------------')
print('\n')

---------------------------------------------------------------------------------------------------------
TREATING MISSING VALUES


Training data:
Number of overall missing values detected before treatment: 2242917.
Number of overall missing values detected during treatment: 2242917.


Test data:
Number of overall missing values detected before treatment: 4271178.
Number of overall missing values detected during treatment: 4271178.


---------------------------------------------------------------------------------------------------------




<a id='categorical_transf'></a>

### Transforming categorical features

#### Creating dummies through one-hot encoding

In [28]:
# Create object for one-hot encoding:
categorical_transf = one_hot_encoding(categorical_features = cat_vars)

# Creating dummies:
categorical_transf.create_dummies(categorical_train = categorical_train,
                                  categorical_test = categorical_test)

# Selected dummies:
dummy_vars = list(categorical_transf.dummies_train.columns)

# Training data:
dummies_train = categorical_transf.dummies_train
dummies_train.index = df_train_scaled.index

# Test data:
dummies_test = categorical_transf.dummies_test
dummies_test.index = df_test_scaled.index

# Dropping original categorical features:
df_train_scaled.drop(cat_vars, axis=1, inplace=True)
df_test_scaled.drop(cat_vars, axis=1, inplace=True)

print('\033[1mNumber of categorical features:\033[0m {}.'.format(len(categorical_transf.categorical_features)))
print('\033[1mNumber of overall selected dummies:\033[0m {}.'.format(dummies_train.shape[1]))
print('\033[1mShape of dummies_train for store ' + str(s) + ':\033[0m ' +
      str(dummies_train.shape) + '.')
print('\033[1mShape of dummies_test for store ' + str(s) + ':\033[0m ' +
      str(dummies_test.shape) + '.')
print('\n')

dummies_train.head()

Number of categorical features: 14.
Number of overall selected dummies: 51.
Shape of dummies_train for store 9098: (7520, 51).
Shape of dummies_test for store 9098: (11218, 51).




,C#BILLINGCITY()#BARUERI,C#BILLINGCITY()#OSASCO,C#BILLINGCITY()#SANTANA_DE_PARNAIBA,C#BILLINGCITY()#SAO_PAULO,C#BROWSER()#CHROME,C#BROWSER()#CHROME_MOBILE,C#BROWSER()#FIREFOX,C#BROWSER()#MOBILE_SAFARI,C#BROWSER()#NA_VALUE,C#BROWSER()#SAFARI,...,C#SELLERID()#31,C#SELLERID()#35,C#SELLERID()#NONE,C#SHIPPINGCITY()#BARUERI,C#SHIPPINGCITY()#OSASCO,C#SHIPPINGCITY()#SANTANA_DE_PARNAIBA,C#SHIPPINGCITY()#SAO_PAULO,C#UTMSOURCELASTCLICK()#EMAIL,C#UTMSOURCELASTCLICK()#FACEBOOK,C#UTMSOURCELASTCLICK()#NA_VALUE
0,0,0,0,1,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
1,0,0,0,1,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
2,0,0,0,1,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,1
3,0,0,0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
4,0,0,0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1


#### Concatenating all features

In [29]:
df_train_scaled = pd.concat([df_train_scaled, dummies_train], axis=1)
df_test_scaled = pd.concat([df_test_scaled, dummies_test], axis=1)

print('\033[1mShape of df_train_scaled for store ' + str(s) + ':\033[0m ' + str(df_train_scaled.shape) + '.')
print('\033[1mShape of df_test_scaled for store ' + str(s) + ':\033[0m ' + str(df_test_scaled.shape) + '.')
print('\n')

df_train_scaled.head()

Shape of df_train_scaled for store 9098: (7520, 2104).
Shape of df_test_scaled for store 9098: (11218, 2104).




,BUREAUBILLCITY(),BUREAUBILLSTATE(),BUREAUDOB(),BUREAUEMAIL(),BUREAUPHONE(),BUREAUPHONEAREACODE(),BUREAUSHIPCITY(),BUREAUSHIPSTATE(),CREDITCARDCOUNTRYSAMEASBILLING(),CREDITCARDCOUNTRYSAMEASSHIPPING(),...,C#SELLERID()#31,C#SELLERID()#35,C#SELLERID()#NONE,C#SHIPPINGCITY()#BARUERI,C#SHIPPINGCITY()#OSASCO,C#SHIPPINGCITY()#SANTANA_DE_PARNAIBA,C#SHIPPINGCITY()#SAO_PAULO,C#UTMSOURCELASTCLICK()#EMAIL,C#UTMSOURCELASTCLICK()#FACEBOOK,C#UTMSOURCELASTCLICK()#NA_VALUE
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0,0,1,0,0,0,1,0,0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0,0,1,0,0,0,1,0,0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0,0,1,0,0,0,1,0,0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0,0,1,0,0,0,1,0,0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0,0,1,0,0,0,1,0,0,1


In [30]:
# Assessing missing values (training data):
num_miss_train = df_train_scaled.isnull().sum().sum() > 0
if num_miss_train:
    print('\033[1mProblem - Number of overall missings detected (training data):\033[0m ' +
          str(df_train_scaled.isnull().sum().sum()) + '.')
    print('\n')

# Assessing missing values (test data):
num_miss_test = df_test_scaled.isnull().sum().sum() > 0
if num_miss_test:
    print('\033[1mProblem - Number of overall missings detected (test data):\033[0m ' +
          str(df_test_scaled.isnull().sum().sum()) + '.')
    print('\n')

<a id='datasets_structure'></a>

### Datasets structure

In [31]:
# Checking consistency of structure between training and test dataframes:
if len(list(df_train_scaled.columns)) != len(list(df_test_scaled.columns)):
    print('\033[1mProblem - Inconsistent number of columns between dataframes for training and test data!\033[0m')

else:
    consistency_check = 0
    
    # Loop over variables:
    for c in list(df_train_scaled.columns):
        if list(df_train_scaled.columns).index(c) != list(df_test_scaled.columns).index(c):
            print('\033[1mProblem - Feature {0} was positioned differently in training and test dataframes!\033[0m'.format(c))
            consistency_check += 1
            
    # Reordering columns of test dataframe:
    if consistency_check > 0:
        ordered_columns = list(df_train_scaled.columns)
        df_test_scaled = df_test_scaled[ordered_columns]

<a id='model_estimation'></a>

## Model estimation

<a id='grids'></a>

### Grids of hyper-parameters

In [32]:
# Declare grid of hyper-parameters:
if method == 'logistic_regression':
    # Default values for hyper-parameters:
    params_default = {'C': 1}

    # Grid of values for hyper-parameters:
    grid_regul = [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.25, 0.3, 0.5, 0.75, 1, 3, 10]

    if random_search:
        # Number of samples from each random distribution of regularization parameter:
        n1 = int(n_samples*round(len([x for x in grid_regul if (x < 0.1)])/len(grid_regul), 2))
        n2 = int(n_samples*round(len([x for x in grid_regul if (x >= 0.1) & (x < 1)])/len(grid_regul), 2)) + 1
        n3 = int(n_samples*round(len([x for x in grid_regul if x >= 1])/len(grid_regul), 2))

        grid_regul = []

        # Loop over random distributions of regularization parameter:
        for d in [uniform(0.0001, 0.1).rvs(n1), uniform(0.1, 1).rvs(n2), uniform(1, 10).rvs(n3)]:
            for x in d:
                grid_regul.append(x)
    
    params = {'C': grid_regul}

elif method == 'GBM':
    # Default values of hyper-parameters:
    params_default = {'subsample': 0.75,
                      'learning_rate': 0.01,
                      'max_depth': 3,
                      'n_estimators': 500}
    
    if random_search:
        # Random distributions of hyper-parameters:
        params = {'subsample': [0.75],
                  'learning_rate': uniform(0.0001, 0.1),
                  'max_depth': randint(1, 5+1),
                  'n_estimators': randint(100, 500+1)}

    else:
        # Grid of values for hyper-parameters:
        params = {'subsample': [0.75],
                  'learning_rate': [0.0001, 0.01, 0.1],
                  'max_depth': [1, 3, 5],
                  'n_estimators': [100, 250, 500]}

<a id='estimations'></a>

### Estimations

In [38]:
# Declare K-folds CV estimation object:
train_test_est = Kfolds_fit(task = 'classification', method = method,
                            metric = 'roc_auc', num_folds = 3,
                            pre_selecting = False, pre_selecting_param = None,
                            random_search = random_search, n_samples = n_samples,
                            grid_param = params, default_param = params_default)

# Running train-test estimation:
train_test_est.run(train_inputs=df_train_scaled.drop(drop_vars, axis=1),
                   train_output=df_train_scaled['y'],
                   test_inputs=df_test_scaled.drop(drop_vars, axis=1),
                   test_output=df_test_scaled['y'])

# Defining best tuning hyper-parameter:
best_params = train_test_est.best_param

# Assessing performance metrics:
test_roc_auc = train_test_est.performance_metrics["test_roc_auc"]
test_prec_avg = train_test_est.performance_metrics["test_prec_avg"]
test_brier = train_test_est.performance_metrics["test_brier"]

Grid estimation progress: [----------------------------------------------] 100%



Running time (K-folds CV estimation): 0.4 minutes.
Start time: 2020-12-31, 17:07:25
End time: 2020-12-31, 17:07:50


---------------------------------------------------------------------
K-folds CV outcomes:
Number of data folds: 3.
Number of samples for random search: 10.
Estimation method: logistic regression.
Metric for choosing best hyper-parameter: roc_auc.
Best hyper-parameters: {'C': 0.3}.
CV performance metric associated with best hyper-parameters: 0.9081.
---------------------------------------------------------------------




--------------------------------------------
Performance metrics evaluated at test data:
test_roc_auc = 0.9639
test_prec_avg = 0.7539
test_brier = 0.0028
--------------------------------------------


Total rununning time: 0.42 minutes.
Start time: 2020-12-31, 17:07:25
End time: 2020-12-31, 17:07:51




<a id='assess_results'></a>

## Assessment of results

In [39]:
end_time = datetime.now()

model_assessment[estimation_id] = {
    'store_id': s,
    'n_orders_train': len(df_train_scaled),
    'n_orders_test': len(df_test_scaled),
    'n_vars': df_train_scaled.drop(drop_vars, axis=1).shape[1],
    'first_date_train': str(df_train_scaled.date.min().date()),
    'last_date_train': str(df_train_scaled.date.max().date()),
    'first_date_test': str(df_test_scaled.date.min().date()),
    'last_date_test': str(df_test_scaled.date.max().date()),
    'avg_order_amount_train': df_train_scaled.order_amount.mean(),
    'avg_order_amount_test': df_test_scaled.order_amount.mean(),
    'log_transform': log_transform,
    'standardize': standardize,
    'method': method,
    'random_search': random_search,
    'n_samples': n_samples,
    'best_param': str(best_params),
    'test_roc_auc': test_roc_auc,
    'test_brier': test_prec_avg,
    'test_prec_avg': test_brier,
    'running_time': str(round((end_time - start_time).seconds/60 , 2)) + ' minutes'
}

if export:
    with open('Datasets/model_assessment.json', 'w') as json_file:
        json.dump(model_assessment, json_file, indent=2)